In [1]:
import pandas as pd 
# ("./data/etl_ymd.csv")# 주말인지 아닌지 
# ("./data/"id_WGS.csv) #위도 경도 데이터
dir_="C://Users/sunkeun_jo/Desktop\data/etl_ymd.csv"

In [2]:
df=pd.read_csv(dir_)
df.head()

,etl_ymd,etl_ymd_dateType,weekday
0,20180101,2018-01-01,0
1,20180102,2018-01-02,1
2,20180103,2018-01-03,2
3,20180104,2018-01-04,3
4,20180105,2018-01-05,4


In [3]:
print(df['etl_ymd'].dtype)

int64


In [4]:
from datetime import datetime 
df['etl_ymd_Type']=df['etl_ymd'].apply(lambda x : str(x)[0:4]+"-"+str(x)[4:6]+"-"+str(x)[6:])
df['etl_ymd_dateType']=df['etl_ymd_Type'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
df.head()

,etl_ymd,etl_ymd_dateType,weekday,etl_ymd_Type
0,20180101,2018-01-01,0,2018-01-01
1,20180102,2018-01-02,1,2018-01-02
2,20180103,2018-01-03,2,2018-01-03
3,20180104,2018-01-04,3,2018-01-04
4,20180105,2018-01-05,4,2018-01-05


In [45]:
df.tail()

,etl_ymd,etl_ymd_dateType,weekday,etl_ymd_Type
360,20181227,2018-12-27,3,2018-12-27
361,20181228,2018-12-28,4,2018-12-28
362,20181229,2018-12-29,5,2018-12-29
363,20181230,2018-12-30,6,2018-12-30
364,20181231,2018-12-31,0,2018-12-31


# 공공 데이터포털

공공데이터 포털에서 기상청18_동네예보_조회서비스_오픈API 서비스 사용 그 중 초단기 실황 조회로 그 당시의 실제 날씨를 찾고자함 

울산광역시 중구 북정동에 위치한 관측소를 기준으로 여러가지 정보들을 추출했습니다.

In [89]:
servicekey="0dK1EIo6UluKLHUFpdCi0cbu2M1yQ0xDrrUwxEoEy%2Fz3ZXptbQ6rQ7KqBmuj1mTyXKglkIhWyCjs8abFAYYdDg%3D%3D"
endpoint="http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey={}&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=152&schListCnt=10&endDt=20181231&endHh=23&startHh=01&startDt=20180101".format(servicekey)

In [90]:
print(endpoint)

http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=0dK1EIo6UluKLHUFpdCi0cbu2M1yQ0xDrrUwxEoEy%2Fz3ZXptbQ6rQ7KqBmuj1mTyXKglkIhWyCjs8abFAYYdDg%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=152&schListCnt=10&endDt=20181231&endHh=01&startHh=01&startDt=20180101


In [98]:
import requests
from bs4 import BeautifulSoup

In [120]:
resp=requests.get(endpoint)
print(resp.status_code)
print(resp.text)


200
<?xml version="1.0" encoding="UTF-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items /><numOfRows>10</numOfRows><pageNo>2911</pageNo><totalCount>8737</totalCount></body></response>



In [174]:
import time 
import pandas as pd 
import requests
from bs4 import BeautifulSoup

servicekey="0dK1EIo6UluKLHUFpdCi0cbu2M1yQ0xDrrUwxEoEy%2Fz3ZXptbQ6rQ7KqBmuj1mTyXKglkIhWyCjs8abFAYYdDg%3D%3D"
j=0
inform_list=[]


while True:
    
    j+=1
    endpoint="http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey={}&numOfRows=10&pageNo={}&dataCd=ASOS&dateCd=HR&stnIds=152&schListCnt=10&endDt=20181231&endHh=23&startHh=01&startDt=20180101".format(servicekey,j)
    resp=requests.get(endpoint)
    time.sleep(3)# 찾을 시간을 줘야합니다....
    if resp.status_code ==200 :
        soup=BeautifulSoup(resp.text)
        inform_=soup.find_all("item")
        for i in inform_:
            if i.find("tm").text < "2019-01-01 00:00":
                inform_list.append([i.find("tm").text,i.find("ta").text,i.find("rn").text,i.find("ws").text,i.find("hm").text,i.find("dsnw").text])
                print(i.find("tm").text)
                print(" 데이터 {}개가 들어갔습니다.".format(len(inform_list)))
            else:
                break
                
        if inform_==[]:
            break
        
    else: 
        break
        
df=pd.DataFrame(inform_list,columns=["시간","기온","강수량","풍속","습도","적설"])
df.to_csv("ulsan_inform.csv",index=False,encoding='utf-8')

2018-12-30 19:00
 데이터 1개가 들어갔습니다.
2018-12-30 20:00
 데이터 2개가 들어갔습니다.
2018-12-30 21:00
 데이터 3개가 들어갔습니다.
2018-12-30 22:00
 데이터 4개가 들어갔습니다.
2018-12-30 23:00
 데이터 5개가 들어갔습니다.
2018-12-31 00:00
 데이터 6개가 들어갔습니다.
2018-12-31 01:00
 데이터 7개가 들어갔습니다.
2018-12-31 02:00
 데이터 8개가 들어갔습니다.
2018-12-31 03:00
 데이터 9개가 들어갔습니다.
2018-12-31 04:00
 데이터 10개가 들어갔습니다.
874번째 페이지의 내용이 들어갔습니다.
2018-12-31 05:00
 데이터 11개가 들어갔습니다.
2018-12-31 06:00
 데이터 12개가 들어갔습니다.
2018-12-31 07:00
 데이터 13개가 들어갔습니다.
2018-12-31 08:00
 데이터 14개가 들어갔습니다.
2018-12-31 09:00
 데이터 15개가 들어갔습니다.
2018-12-31 10:00
 데이터 16개가 들어갔습니다.
2018-12-31 11:00
 데이터 17개가 들어갔습니다.
2018-12-31 12:00
 데이터 18개가 들어갔습니다.
2018-12-31 13:00
 데이터 19개가 들어갔습니다.
2018-12-31 14:00
 데이터 20개가 들어갔습니다.
875번째 페이지의 내용이 들어갔습니다.
2018-12-31 15:00
 데이터 21개가 들어갔습니다.
2018-12-31 16:00
 데이터 22개가 들어갔습니다.
2018-12-31 17:00
 데이터 23개가 들어갔습니다.
2018-12-31 18:00
 데이터 24개가 들어갔습니다.
2018-12-31 19:00
 데이터 25개가 들어갔습니다.
2018-12-31 20:00
 데이터 26개가 들어갔습니다.
2018-12-31 21:00
 데이터 27개가 들어갔습니다.
2018-12-31 22:00
 

In [166]:
df=pd.DataFrame(inform_list,columns=["시간","기온","강수량","풍속","습도","적설"])
df.to_csv("ulsan_inform2.csv",index=False,encoding='utf-8')

In [2]:
import pandas as pd
df=pd.read_csv("ulsan_inform2.csv")
df.tail()

FileNotFoundError: File b'ulsan_inform2.csv' does not exist

In [161]:
stop=True
stop

True

# Introduction

공유 자전거 재배치 전략 수립 연구

- [ ] DB 구축 
- [ ] 유동인구 경로 예측
- [ ] 재배치 전략 수립


# 0. DB 구축

https://github.com/sugyeong-jo/Bicycle_sharing/blob/master/data/DB.md

- [ ] 4월 13일

## 0-1. pydec 
- 관련 예제 정리 
- [ ] 4월 13일


# 1. 유동인구 경로 예측

## 1-1. 유동인구 수 예측
- 뉴럴 네트워크를 이용한 유동인구 수 예측
- pytorch 이용 예정
- [ ] 4월 20일

## 1-2. 유동인구 경로 예측
- gravity spatial interaction model
- key word: gravity spatial interaction model, OD flow, Dynamic OD estimation, ...
- [ ] 4월 27일

# 2. 경로에 따른 배터리 예측 시뮬레이션
- 시뮬레이션 논문 찾고 구현
- [ ] 5월 4일

# 3. 재배치 전략 수립
- Dynamic pricing model
    > Gig economy!

    > 택배
- Vehicle routing problem 
- Equilibrium model
- [ ] 매 중 2개 이상 찾고 정리하기
- [ ] 5월 11일 간단한 예제 만들기
- [ ] 5월 17일 두 가지에 대한 간단한 예제 만들기
- [ ] 5월 24일 equilibrium model 예제 만들기





